In [2]:
# Import packages
import http.client
import json
import pandas as pd
from datetime import datetime
import numpy as np
import time
import pickle

In [113]:
# Api keys
eda_api_key = "thp5rpgrbegb8hkc7zrapy36"
mds_api_key = "fqe4f9fyp8m84mg679xu7dnj"
prod_api_key = "gUjaqGzSOH2evg8aqQF67aSOmZDLbLpU2ou03nxh"

# Variable to count api calls
global CALLS
CALLS = 0

# Make an API call given get, return a json object
def send_req(get, func):
    global CALLS
    # Connect to SR API
    conn = http.client.HTTPSConnection("api.sportradar.com")
    conn.request("GET", get)
    with conn.getresponse() as res:
        # Check if the response status is OK (200)
        if res.status == 200:
            data = res.read().decode("utf-8")
            
            # Check if data is not empty before decoding
            if data:
                CALLS += 1
                return json.loads(data)
            else:
                print("Error: Empty response data: " + func)
                return None
        else:
            print(f"Error: Failed request with status code {res.status}: {res.read().decode()}:" + func)
            return None

# Display json object neatly
def pp(json_obj):
    print(json.dumps(json_obj, indent=4))

In [4]:
with open('data/gs_matches_men.pkl', 'rb') as f:
    gs_matches_m = pickle.load(f)

with open('data/gs_matches_women.pkl', 'rb') as f:
    gs_matches_w = pickle.load(f)

In [122]:
# Query sport event timeline
def rq_sport_event_timeline(event, api_key, type_ = "trial"):
    pre = "/tennis/"+type_+"/v3/en/sport_events/"
    post = "/timeline.json"
    get = pre+event+post+"?api_key="+api_key
    return send_req(get,"rq_sport_event_timeline")

#timeline_m = rq_sport_event_timeline(gs_matches_m[0], eda_api_key, "trial")
#timeline_w = rq_sport_event_timeline(gs_matches_w[1], eda_api_key, "trial")
#timeline_p = rq_sport_event_timeline(gs_matches_w[1], prod_api_key, "production")
#pp(timeline0)


In [116]:
pp(timeline_p)

{
    "generated_at": "2024-02-08T19:57:22+00:00",
    "sport_event": {
        "id": "sr:sport_event:42023447",
        "start_time": "2023-07-03T10:00:00+00:00",
        "start_time_confirmed": true,
        "sport_event_context": {
            "sport": {
                "id": "sr:sport:5",
                "name": "Tennis"
            },
            "category": {
                "id": "sr:category:6",
                "name": "WTA"
            },
            "competition": {
                "id": "sr:competition:2559",
                "name": "Wimbledon Women Singles",
                "parent_id": "sr:competition:2553",
                "type": "singles",
                "gender": "women",
                "level": "grand_slam"
            },
            "season": {
                "id": "sr:season:95309",
                "name": "Wimbledon Women Singles 2023",
                "start_date": "2023-06-26",
                "end_date": "2023-07-16",
                "year": "2023",
         

In [234]:
def str_to_time(str):
    return datetime.fromisoformat(str)


def play_by_play(timeline):

    # for each event in the timeline
    game_time = None
    game_lengths = []
    points_per_game = []
    in_game = False
    suspended = False
    suspended_start = 0
    suspended_time = 0
    in_game = 0
    for time in timeline['timeline']:
        if time['type']!='point':
            curr_time = str_to_time(time['time'])

            if in_game!=0: # game ended, add points to list
                points_per_game.append(in_game+1) # +1 for the winning point
                in_game = 0

            if time['type']=='match_suspended':
                suspended=True
                suspended_start = curr_time

            elif time['type']=='match_resumed':
                suspended=False
                suspended_time = (curr_time-suspended_start).total_seconds()/60

            # Compute game length
            elif time['type'] in ['period_score','period_start'] and suspended ==False: # start of a set or game
                if game_time!=None:
                    point_length = (curr_time-game_time).total_seconds()/60
                    if point_length < 20: # there are no twenty minute points
                        game_lengths.append(point_length)
                    else:
                        point_length = (curr_time-game_time).total_seconds()/60-suspended_time
                        game_lengths.append(point_length)
                    if suspended_time!=0:
                        suspended_time=0

                game_time = curr_time # start of next step time

            continue
        else:
            in_game+=1
    game_lengths.append(0) # extra zero indicating end of the set/match

    sum_set = 0
    sum_game = 0
    num_games = 0
    set_lengths = []
    avg_game_lengths_per_set = []

    for point in game_lengths:
        if point<.1: # sometimes time between end of game and point recorded is not zero
            set_lengths.append(sum_set)
            sum_set=0
            avg_game_lengths_per_set.append(sum_game/num_games)
            num_games=0
            sum_game=0
        else:
            sum_set+=point
            sum_game+=point
            num_games+=1
    
    game_lengths = [x for x in game_lengths if x>.1]

    avg_set_length = np.nanmean(set_lengths)
    avg_game_length = np.nanmean(game_lengths)
    avg_points_per_game = np.nanmean(points_per_game)
    match_length = np.sum(set_lengths)

    return [avg_set_length, set_lengths, avg_game_length, avg_game_lengths_per_set, avg_points_per_game, match_length]

In [233]:
def get_event_stats(timeline):
    if timeline['sport_event_status']['match_status'] == 'retired':
        return None
    
    event_id = timeline['sport_event']['id']
    stats = [pd.DataFrame(),pd.DataFrame()]
    context = timeline['sport_event']['sport_event_context']

    for i in range(len(stats)):
        comp_info = timeline['statistics']['totals']['competitors'][i]
        stats_i = pd.json_normalize(comp_info['statistics'])
        stats_i.insert(0, 'event_id', event_id)
        stats_i.insert(1, 'player_id', comp_info['id'])
        stats_i.insert(2, 'name', comp_info['name'])
        try:
            seed = timeline['sport_event']['competitors'][i]['seed']
        except:
            seed = None
        stats_i.insert(3, 'seed', seed)
        
        stats[i] = stats_i

    merged = pd.merge(stats[0], stats[1], how='outer', left_on = 'event_id', right_on='event_id', suffixes=("_home", "_away"))

    # start date of competitions
    merged['date'] = context['season']['start_date']
    #timeline['sport_event']['start_time']

    # Name of competition
    merged['competition'] = context['competition']['name']

    # Men or womens match
    merged['men'] = (context['competition']['gender']=="men")

    # Which round?
    merged['round'] = context['round']['name']

    # How many sets
    merged['best_of'] = context['mode']['best_of']

    # Who won?
    merged['win_1'] = timeline['sport_event_status']['home_score']>timeline['sport_event_status']['away_score']

    # Get play by play
    [avg_set_length, set_lengths, avg_game_length, avg_game_lengths_per_set, avg_points_per_game, match_length] = play_by_play(timeline)

    merged["avg_set_length"] = avg_set_length
    merged["avg_game_length"] = avg_game_length
    merged["avg_points_per_game"] = avg_points_per_game
    merged['match_length'] = match_length

    # Avgs per set
    for set_ in range(len(set_lengths)):
        merged["avg_game_length_set"+str(set_+1)] = avg_game_lengths_per_set[set_]
        merged["set"+str(set_+1)+"_length"] = set_lengths[set_]

    # Getting scores for each set
    i = 1
    for set_ in timeline['sport_event_status']['period_scores']:
        merged['set'+str(i)+"_diff"] = set_['home_score']-set_['away_score']
        merged['set'+str(i)+'_games'] = set_['home_score']+set_['away_score']
        i+=1

    return merged
    

In [191]:
test = "sr:sport_event:42024017"
timeline_test = rq_sport_event_timeline(test, prod_api_key,"production")

In [199]:
timeline_test

{'generated_at': '2024-02-08T20:01:25+00:00',
 'sport_event': {'id': 'sr:sport_event:42024017',
  'start_time': '2023-07-05T15:15:00+00:00',
  'start_time_confirmed': True,
  'sport_event_context': {'sport': {'id': 'sr:sport:5', 'name': 'Tennis'},
   'category': {'id': 'sr:category:3', 'name': 'ATP'},
   'competition': {'id': 'sr:competition:2555',
    'name': 'Wimbledon Men Singles',
    'parent_id': 'sr:competition:2553',
    'type': 'singles',
    'gender': 'men',
    'level': 'grand_slam'},
   'season': {'id': 'sr:season:95307',
    'name': 'Wimbledon Men Singles 2023',
    'start_date': '2023-06-26',
    'end_date': '2023-07-16',
    'year': '2023',
    'competition_id': 'sr:competition:2555'},
   'stage': {'order': 2,
    'type': 'cup',
    'phase': 'stage_1_playoff',
    'start_date': '2023-07-03',
    'end_date': '2023-07-16',
    'year': '2023'},
   'round': {'name': 'round_of_128'},
   'groups': [{'id': 'sr:cup:142313',
     'name': '2023 Wimbledon, London, Great Britain'}],


In [231]:
test_df = get_event_stats(timeline_test)
test_df

[2.716666666666667, 3.0, 2.55, 3.3, 2.316666666666667, 4.816666666666666, 3.6, 2.933333333333333, 3.2, 4.25, 2.4166666666666665, 6.516666666666667, 0.0, 4.566666666666666, 3.75, 10.883333333333333, 4.0, 2.183333333333333, 2.8666666666666667, 7.583333333333333, 3.933333333333333, 3.966666666666667, 0.016666666666666666, 3.7333333333333334, 3.066666666666667, 2.05, 3.6, 3.566666666666667, 3.8666666666666667, 3.75, 0]
[41.61666666666667, 43.733333333333334, 23.633333333333336]


,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,double_faults_home,...,avg_game_length_set2,set2_length,avg_game_length_set3,set3_length,set1_diff,set1_games,set2_diff,set2_games,set3_diff,set3_games
0,sr:sport_event:42024017,sr:competitor:503814,"Mochizuki, Shintaro",None,3,20,11,8,1,2,...,4.859259,43.733333,3.37619,23.633333,-2,12,-3,9,-5,7


In [235]:
# list to one big dataframe of match statistics
mens = []
count = 0
for match in gs_matches_m:
    timeline = rq_sport_event_timeline(match, prod_api_key, "production")
    mens.append(get_event_stats(timeline))
    count=count+1
    if count%25==0:
        print("{} matches extracted.".format(count))

mens_df = pd.concat(mens).reset_index()
mens_df = mens_df.drop(columns=['index'])
mens_df

,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,double_faults_home,...,set1_diff,set1_games,set2_diff,set2_games,set3_diff,set3_games,set4_diff,set4_games,set5_diff,set5_games
0,sr:sport_event:42023937,sr:competitor:87690,"Thompson, Jordan",None,16,29,13,7,3,2,...,-4,8,-4,8,2,10,1.0,13.0,3.0,9.0
1,sr:sport_event:42023949,sr:competitor:359602,"Musetti, Lorenzo",14,5,13,11,10,5,4,...,3,9,5,7,2,12,NaN,NaN,NaN,NaN
2,sr:sport_event:42023961,sr:competitor:257721,"Baez, Sebastian",None,3,18,24,13,2,7,...,-1,13,3,9,-3,9,-1.0,13.0,NaN,NaN
3,sr:sport_event:42023963,sr:competitor:658475,"van Assche, Luca",None,6,21,15,4,2,6,...,1,13,-2,10,-4,8,-2.0,10.0,NaN,NaN
4,sr:sport_event:42023965,sr:competitor:106755,"Rublev, Andrey",7,6,13,11,4,5,1,...,3,9,2,12,2,10,NaN,NaN,NaN,NaN


In [112]:
# list to one big dataframe of match statistics
womens = []
count = 0
for match in gs_matches_w[31:32]:
    print(timeline)
    timeline = rq_sport_event_timeline(match, mds_api_key)
    womens.append(get_event_stats(timeline))
    count=count+1
    if count%25==0:
        print("{} matches extracted.".format(count))

womens_df = pd.concat(womens).reset_index()
womens_df = womens_df.drop(columns=['index'])
womens_df

None
[2.3833333333333333, 6.316666666666666, 2.9, 4.95, 3.2333333333333334, 6.916666666666667, 4.783333333333333, 2.566666666666667, 4.016666666666667, 0.0, 5.05, 2.2, 3.433333333333333, 4.2, 2.533333333333333, 3.316666666666667, 10.450000000000045, 2.6, 4.466666666666667]
[38.06666666666667, 38.25000000000004]


,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,double_faults_home,...,round,best_of,win_1,avg_set_length,avg_game_length,avg_points_per_game,set1_diff,set1_games,set2_diff,set2_games
0,sr:sport_event:42023387,sr:competitor:234150,"Potapova, Anastasia",22,0,8,8,2,6,4,...,round_of_128,3,True,38.158333,4.239815,6.368421,3,9,3,9


In [56]:
womens_df.to_csv("data/womens_df.csv", index=False)

In [57]:
womens_past = womens_df

Break points

In [ ]:

# Break points
# Is it a break point?
# if is_break_point(int(time['home_score']), int(time['away_score']), time['server']):
#    if int(time['home_score'])>int(time['away_score']):
#       break_point_home+=1
#    else:
#       break_point_away+=1

# def is_break_point(home_score, away_score, server):

#     if home_score==away_score: # score is tied
#         return False
#     elif (home_score>away_score and server=='home') or (away_score>home_score and server=='away'): # server is winning
#         return False
#     elif not (home_score==40 or away_score==40): # not at the end of the game
#         return False
#     else:
#         return True

# # Compute set length
# if time['type'] in ['period_start','match_ended'] and suspended==False: # beginning/ends of sets
#     if "period_name" in time and time['period_name']=='suspended':
#         suspended = True
#         continue # Neglecting any part of the game that follows a suspension (I think this can be fixed!!!!! but keep for now)

#     curr_time = str_to_time(time['time'])
#     if set_time != None:
#         set_lengths.append( (curr_time-set_time).total_seconds() /60) # subtracting two set times to get set length in minutes
#     set_time = curr_time # start of next step time